In [1]:
# Install required packages
!pip install transformers flask flask-cors pyngrok --quiet
!pip install sentence-transformers scikit-learn --quiet
!pip install sacremoses --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 37.4 MB/s eta 0:00:00


# Load Models

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT-Large")
model = AutoModelForCausalLM.from_pretrained("microsoft/BioGPT-Large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/566k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.29G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

biobert = SentenceTransformer('pritamdeka/S-BioBert-snli-multinli-stsb', device=device)

# Document Embeddings & Retriver

In [ ]:
docs = []
doc_embedings = []

In [ ]:
def add_doc(text):
    # Embed the incoming doc using BioBERT
    embedding = biobert.encode([text], convert_to_tensor=True)[0].cpu().numpy()

    # Store in memory
    docs.append(text)
    doc_embeddings.append(embedding)

    print(f"✅ Added document. Total docs: {len(docs)}")

In [ ]:
def find_most_similar_doc(query, top_k=1):
    if not doc_embeddings:
        return "❌ No documents in memory."

    # Embed the user query
    query_embedding = biobert.encode([query], convert_to_tensor=True)[0].cpu().numpy()

    # Compute cosine similarities
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

    # Get the top-k most similar docs
    top_indices = similarities.argsort()[-top_k:][::-1]

    results = []
    for idx in top_indices:
        results.append({
            "doc": docs[idx],
            "similarity": float(similarities[idx])
        })

    return results

# API

In [ ]:
!pip install flask flask-cors

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

@app.route('/embed-doc', methods=['POST'])
def embed_doc():
    data = request.get_json()
    text = data.get("text", "")
    if not text:
        return jsonify({"error": "Missing 'text' field."}), 400

    add_doc(text)
    return jsonify({"message": "✅ Document embedded.", "total_docs": len(docs)})

@app.route('/query', methods=['POST'])
def query():
    data = request.get_json()
    query = data.get("query", "")
    if not query:
        return jsonify({"error": "Missing 'query' field."}), 400

    results = find_most_similar_doc(query, top_k=1)
    return jsonify(results)

In [ ]:
!pip install pyngrok

In [ ]:
token = input('Enter ngrok auth token: ')

In [ ]:
from pyngrok import ngrok

# Replace with your token from ngrok.com
ngrok.set_auth_token(token)

In [ ]:
# Run Flask app in background
import threading

def run_flask():
    from flask_cors import CORS
    CORS(app)
    app.run(port=5000)

thread = threading.Thread(target=run_flask)
thread.start()

# Start ngrok tunnel
from pyngrok import ngrok

public_url = ngrok.connect(5000)
print("🚀 Public URL:", public_url)

In [ ]:
import os
import signal
from pyngrok import ngrok, conf

# Kill ngrok tunnel
try:
    ngrok.kill()
    print("✅ Ngrok tunnel closed.")
except Exception as e:
    print("⚠️ Error closing ngrok:", e)

# Kill Flask server (running in background if any)
try:
    os.kill(os.getpid(), signal.SIGKILL)
except:
    pass  # Colab will just stop the kernel